In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#important library imports
import numpy as np
import cv2 
import os
from google.colab.patches import cv2_imshow
from scipy.signal import find_peaks, peak_prominences
from google.colab import files
from IPython.display import clear_output

In [ ]:
!tar xvzf "/content/drive/MyDrive/NLP/OnlineKhatt.tar.gz" -C "/content/"
#upload your line images in a zipped folder called OnlineKhatt
!unzip "/content/drive/MyDrive/NLP/OnlineKhatt.tar.gz" -d "/content/"
#clear console after inflating
clear_output()

In [ ]:
# list the file names inside the folder called OnlineKhatt
filenames=os.listdir('/content/OnlineKhatt/Images')
# strip filenames from the file extension for further use
filenames_split=[filename.replace('.tif', '') for filename in filenames]

In [ ]:
def directionalHistogram(img, direction='H'):
  # A function to calculate the intensity histograms of an image in x and y directions
    (w,h) = img.shape
    sum = []
    pixel_count=0
    if(direction=='H'):
        for j in range(w-1):
          for i in range(h-1):
            pixel=img[j,i]
            if(pixel==255):
              pixel_count+=1
          sum.append(pixel_count)
          pixel_count=0
    else:
       for j in range(h-1):
          for i in range(w-1):
            pixel=img[i,j]
            if(pixel==255):
              pixel_count+=1
          sum.append(pixel_count)
          pixel_count=0
    return sum

def cropLineToWords(viable_sequences, image):
  #Given a line image and the cutpositions, this functions return the images
  #of the words contained in a line
  (w,h) = image.shape
  words=[]
  for i in range(len(viable_sequences)):
    if(i >0 and i< len(viable_sequences)):
      words.append(image[0:w-1, viable_sequences[i-1]:viable_sequences[i]])
    elif(i== len(viable_sequences)-1):
      words.append(image[0:w-1, viable_sequences[i]:len(viable_sequences)])
  return words

def removeSpaces(words):
  words_without_spaces=[]
  for i in range(len(words)):
    if(np.sum(words[i][:,:]>0)):
      words_without_spaces.append(words[i])
  return words_without_spaces  

In [ ]:
#make a duirectory that will hold the output words
!mkdir words

#loop over all the OnlineKhatt in your line images folder
for m in range(len(filenames)):

  path='/content/OnlineKhatt/Images/'+filenames[m]
  filename=filenames_split[m]
  words=[]

  #read the line image in grey-scale
  img=cv2.imread(path, 0)
  #get dimensions of the image
  (w,h) = img.shape
  #compute the intensity histogram in the y-direction
  hist_vertical=directionalHistogram(img, direction='V')
  #find the locations where the vertical histogram is zero (background spaces between words)
  zero_sites=np.where(np.asarray(hist_vertical)==0)
  zero_sites=zero_sites[0]

  sequences=[]
  sequence_start=0

  #get the start and end of zero sequences in the vertical histogram
  for i in range(1,len(zero_sites)):
    last_zero=zero_sites[i-1]
    current_zero=zero_sites[i]
    if(current_zero!=last_zero+1):
      sequence_end=last_zero
      sequences.append([sequence_start,sequence_end])
      sequence_start=current_zero
    if(current_zero==last_zero+1 and i==len(zero_sites)-1):
      sequence_start=sequence_end
      sequence_end=current_zero
      sequences.append([sequence_start,sequence_end])
  sequence_lengths=[]
  for i in range(len(sequences)):
     sequence_lengths.append(sequences[i][1]-sequences[i][0]+1)
  

  #Threshold the size of the zero sequences (whether it is big enough to consider it as
  # an interword spacing or small enough to consider as intraword spacing)
  sequence_ratio=np.asarray(sequence_lengths)/w
  average_sequence_length=np.sum(sequence_lengths[1:len(sequence_lengths)-1])/len(sequence_lengths)
  viable_sequences=[]
  overlap_factor=0.75*average_sequence_length
  viable_sequences_unrolled=[]

  for i in range(len(sequences)):
      if(sequence_lengths[i]>=average_sequence_length-overlap_factor):
        viable_sequences.append(sequences[i])
        viable_sequences_unrolled.append(sequences[i][0])
        viable_sequences_unrolled.append(sequences[i][1])  

  viable_sequences_unrolled.append(-1)
  if(viable_sequences_unrolled[0]!=0):
        viable_sequences_unrolled=[0]+viable_sequences_unrolled
  words.append(cropLineToWords(viable_sequences_unrolled, img))

  ordered_words=[]

#remove the spaces (word images with blank black background)
  for i in range(len(words[0])):
    word=words[0][i]
    sum=np.sum(words[0][i][:,:])
    if(sum):
      ordered_words.append(word)
    else:
      ordered_words.append('space')

#save word images to the word directory
  count=0
  for i in range(len(ordered_words)):
      if(not type(ordered_words[i]) is str):
        count+=1
        cv2.imwrite("/content/words/"+filename.split('.')[0]+'_word'+str(count)+".tif", ordered_words[i])
        with open("/content/words/"+filename.split('.')[0]+'_word'+str(count)+".txt", 'w') as fid: # 'w' creates a new file
          fid.write('')

#download word directory as a zipped folder
!zip -r /content/words.zip /content/words 
!mv /content/words.zip /content/drive/MyDrive/NLP/
# files.download("/content/words.zip") 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in double_scalars


Streaming output truncated to the last 5000 lines.
  adding: content/words/OnlineKHATT_A0260_3_6_word1.tif (deflated 6%)
  adding: content/words/OnlineKHATT_A0060_3_4_word1.tif (deflated 11%)
  adding: content/words/OnlineKHATT_A0728_3_5_word1.txt (stored 0%)
  adding: content/words/OnlineKHATT_A0131_2_3_word1.tif (deflated 5%)
  adding: content/words/OnlineKHATT_A0080_2_3_word1.tif (deflated 4%)
  adding: content/words/OnlineKHATT_A0296_2_3_word1.tif (deflated 4%)
  adding: content/words/OnlineKHATT_A0283_3_3_word1.txt (stored 0%)
  adding: content/words/OnlineKHATT_A0964_2_4_word1.tif (deflated 7%)
  adding: content/words/OnlineKHATT_A0050_2_1_word1.txt (stored 0%)
  adding: content/words/OnlineKHATT_A0882_3_6_word1.txt (stored 0%)
  adding: content/words/OnlineKHATT_A0703_2_5_word1.tif (deflated 7%)
  adding: content/words/OnlineKHATT_A0994_2_3_word1.tif (deflated 6%)
  adding: content/words/OnlineKHATT_A0434_3_2_word1.tif (deflated 10%)
  adding: content/words/OnlineKHATT_A0406_2_4

FileNotFoundError: ignored